In [2]:
import sys
import itertools
import os

import numpy as np
import pandas as pd
import scipy
from scipy.signal import savgol_filter

import neuroseries as nts

from progressbar import ProgressBar

import time
import matplotlib.pyplot as plt


import seaborn as sns 
import bk.load
import bk.signal
import bk.plot
import importlib; 
importlib.reload(bk.signal)
importlib.reload(bk.plot)
%matplotlib qt
import basefunction.anass_load as at

In [8]:
# path = "Z:\Rat08\Rat08-20130713"
bk.load.path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713'

# bk.load.current_session(path,'/media/billel/GGLab-01-BK/DATA/All-Rats/Billel/session_indexing.csv')

os.chdir(bk.load.path)
bk.load.session = 'Rat08-20130713'

laps = scipy.io.loadmat(bk.load.session+'-LapType.mat')
run = scipy.io.loadmat(bk.load.session + '-TrackRunTimes.mat')
runInt = scipy.io.loadmat('runintervals.mat')['runintervals']
runInt = nts.IntervalSet(runInt[:,0],runInt[:,1],time_units='s')

danger_laps = laps['aplaps'][0]['run'][0]
danger_laps = nts.IntervalSet(danger_laps[:,0],danger_laps[:,1],time_units = 's')

safe_laps = laps['safelaps'][0]['run'][0]
safe_laps = nts.IntervalSet(safe_laps[:,0],safe_laps[:,1],time_units = 's')

trackruntimes = run['trackruntimes']
trackruntimes = nts.IntervalSet(trackruntimes[:,0],trackruntimes[:,1],time_units = 's')

laps = {'run':trackruntimes,'danger':danger_laps,'safe':safe_laps}
states = bk.load.states()

# lfp = bk.load.loadLFP(bk.load.path+ '\\' + bk.load.session+".lfp",166,13)
lfp = pd.Series(dtype = 'object')

inter = runInt

for run in inter.as_units('s').iloc:
    lfp_ = bk.load.lfp(np.floor(run['start']),np.ceil(run['end']),166,13)
    lfp = lfp.append(lfp_)
    
lfp = nts.Tsd(lfp[~lfp.index.duplicated(keep='first')])
pos = bk.load.pos()

In [179]:
del states['drowsy']

lfps = {}
for lap in laps:
    lfp_ = lfp.restrict(laps[lap])
    lfps.update({lap:lfp_})

for state in states:
    lfp_ = lfp.restrict(states[state])
    lfps.update({state:lfp_})

In [10]:
filtered = bk.signal.passband(lfp,5,10)
xa = scipy.signal.hilbert(filtered)
xa = nts.Tsd(np.array(filtered.index),xa)
# x_power = scipy.stats.zscore(np.abs(xa.values)**2)
x_power = (np.abs(xa.values)**2)
# g = scipy.signal.gaussian(125,5)
x_power = np.convolve(x_power,g,'same')
x_power = nts.Tsd(np.array(filtered.index),x_power)
pos['power'] = x_power.reindex(pos.index)
# pos['power'] = position_power.restrict(laps['run'])
a = np.histogram2d(pos.restrict(inter)['x'],pos.restrict(inter)['y'],weights=pos.restrict(inter)['power'],bins = [50,20])
occupancy = np.histogram2d(pos.restrict(inter)['x'],pos.restrict(inter)['y'],[50,20])

plt.figure()
plt.subplot(2,1,1)
plt.imshow(a[0].T/occupancy[0].T)
# plt.imshow(occupancy[0].T)
plt.colorbar()
plt.subplot(2,1,2)
plt.plot(x_power.as_units('s'))
bk.plot.intervals(inter,'orange')

# x_phase = np.angle(xa.values)
# x_phase = nts.Tsd(np.array(filtered.index),x_phase)

# x_rem = x_power.restrict(states['Rem'])
# x_sws = x_power.restrict(states['sws'])

# f_rem = scipy.signal.savgol_filter(x_rem.values,3001,2)
# f_sws = scipy.signal.savgol_filter(x_sws.values,3001,2)


# plt.plot(x_rem.as_units('s'))
# plt.plot(x_sws.as_units('s'))

/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [286]:

# plt.plot(x_power.restrict(laps['run']))
# plt.plot(pos.restrict(laps['run']))
# plt.plot(position_power_run)

In [287]:
pos['power'].index

Int64Index([    9088000,     9113600,     9139200,     9164800,     9190400,
                9216000,     9241600,     9267200,     9292800,     9318400,
            ...
            22679756800, 22679782400, 22679808000, 22679833600, 22679859200,
            22679884800, 22679910400, 22679936000, 22679961600, 22679987200],
           dtype='int64', name='Time (us)', length=885173)

-5001

/home/billel/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [40]:
p = ['run','safe','danger']

for l in p:
    print(l)
    f, Pxx_den = scipy.signal.welch(lfps[l], 1250,nperseg=1024,nfft=1250*10,scaling='density')
    plt.plot(f, Pxx_den)
    plt.xlim(0,100)
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD [V**2/Hz]')
    plt.show()
plt.xlim(2,20)
plt.legend([i for i in p])

run
safe
danger


In [353]:
runInt

,start,end
0,206,9405


In [68]:
plt.plot(position_power.as_units('s'))
bk.plot.intervals(laps['run'],'black')

In [282]:

def thetaMap(lfp,pos):
    filt_lfp = bk.signal

In [283]:
path = bk.load.sessions()['Path'][5]
bk.load.current_session(path)

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


True

In [30]:
lfp = bk.load.loadLFP(bk.load.path+ '\\' + bk.load.session+".lfp",166,101)

In [287]:
states = bk.load.states(bk.load.session)

In [288]:
sub = plt.subplot(3,1,1)
plt.plot(lfp.as_units('s')[3200:3400])
plt.plot(filtered.as_units('s')[3200:3400])
bk.plot.intervals(states['Rem'],'orange')
bk.plot.intervals(states['sws'],'grey')
plt.subplot(3,1,2,sharex = sub)
plt.plot(x_power.as_units('s')[3200:3400])
plt.subplot(3,1,3,sharex = sub)
plt.plot(x_phase.as_units('s')[3200:3400])
# bk.plot.intervals(states['Rem'],'orange',time_units='us')
# bk.plot.intervals(states['sws'],'grey',time_units='us')

In [289]:
f,t,Sxx = scipy.signal.spectrogram(lfp.as_units('s'),fs=1250,mode= 'psd',nperseg=2500,noverlap=1250)
mask = (3000<t) & (t<3500)
plt.pcolormesh(t[mask], f, Sxx[:,mask])
plt.clim(0,20000)
plt.ylim(0,80)
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.show()

In [55]:
pos.restrict(laps['run'])

TypeError: float() argument must be a string or a number, not 'numpy.dtype'

,x,y
Time (us),,
9113600,118.7875,43.78260
9139200,119.4884,42.54936
9164800,120.1678,41.31440
9190400,120.8730,39.90529
9216000,121.5782,38.49618
...,...,...
22670515200,234.7800,37.83699
22670540800,234.7800,38.18400
22670566400,234.7800,38.39556


In [32]:
lfp_rem = lfp.restrict(states['Rem'])

In [355]:
# f_rem = np.convolve(x_rem.values,k,'same')
# f_sws = np.convolve(x_sws.values,k,'same')


In [33]:
sub = plt.subplot(3,1,1)
plt.plot(lfp_rem)
f = bk.signal.passband(lfp_rem,5,10)
plt.plot(bk.signal.passband(lfp_rem,5,10))
plt.subplot(3,1,2 ,sharex = sub)
xa = scipy.signal.hilbert(f)
xa = nts.Tsd(np.array(f.index),xa)
x_phase = np.angle(xa.values)
x_phase = nts.Tsd(np.array(f.index),x_phase)

x_power = 
plt.plot(x_phase)

In [6]:
states

{'Rem':           start          end
 0    1927000000   2013000000
 1    2430000000   2603000000
 2    3276000000   3371000000
 3    3842000000   3887000000
 4    3917000000   3969000000
 5    3993000000   4023000000
 6    4530000000   4567000000
 7    4677000000   4838000000
 8    4864000000   4920000000
 9    5427000000   5488000000
 10   6070000000   6206000000
 11   7312000000   7374000000
 12   7760000000   7787000000
 13   7910000000   8030000000
 14   8798000000   8852000000
 15   9334000000   9402000000
 16  16146000000  16405000000
 17  16766000000  16787000000
 18  17260000000  17363000000
 19  17430000000  17496000000
 20  17811000000  17851000000
 21  18077000000  18131000000
 22  18529000000  18614000000
 23  18980000000  19084000000
 24  19168000000  19186000000
 25  19449000000  19545000000
 26  19772000000  19817000000
 27  21287000000  21358000000
 28  21630000000  21674000000
 29  21851000000  21953000000,
 'sws':           start          end
 0    1448000000   169700